In [1]:
import paramiko
import json
from urllib.parse import urlparse


/home/labuser/miniconda3/envs/benchq/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
## read in a problem_instance.<uuid>.json file


# the file listed is just one example:
problem_instance_path = "problem_instance.fe_red.a8895776-3583-4884-fbc7-d6f9df21a7ac.json"

with open(problem_instance_path, "r") as f:
    problem_instance = json.load(f)


# locate the first (of possibly multiple) Hamiltonian files related to the problem instance
url = problem_instance["instance_data"][0]["instance_data_object_url"]

print(url)
parsed_url = urlparse(url)
Hamiltonian_file_name = parsed_url.path.split("/")[-1]
print(f"\n\nwe will fetch the file: {Hamiltonian_file_name}")

sftp://sftp.l3harris.com/gsee/fe_red.6acme.47d5abd7-c022-4140-4e2c-7918f7d044d8.fcidump.gz


we will fetch the file: fe_red.6acme.47d5abd7-c022-4140-4e2c-7918f7d044d8.fcidump.gz


In [3]:
## define the function to perform the SFTP fetch:

def fetch_file_from_sftp(
        url=None,
        local_path=None,
        ppk_path=None,
        username=None,
        port=None
): 


    parsed_url = urlparse(url)
    hostname = parsed_url.hostname
    remote_path = parsed_url.path.lstrip("/")

    try:
        # Create an SSH client
        with paramiko.SSHClient() as client:
            client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            
            # Connect using the private key file (.ppk)
            client.connect(
                hostname=hostname, 
                port=port, 
                username=username, 
                key_filename=ppk_path
            )

            # Open an SFTP session
            with client.open_sftp() as sftp:
                sftp.get(remote_path, local_path)

        print(f"File fetched successfully from {hostname}")
    except Exception as e:
        print(f"Error: {e}")


In [4]:
## call the function to perform the SFTP fetch:

# standard SCP/SFTP port.
port = 22 

# path to where your key file is!
ppk_path = "/home/labuser/.ssh/darpa-qb-key.ppk"

# your username should match the key file!
username = "darpa-qb"

# call the function:
fetch_file_from_sftp(
    url=url, # obtained URL from problem_instance.<uuid>.json above
    username=username,
    ppk_path=ppk_path, 
    local_path=Hamiltonian_file_name, # name that the file will be saved as
    port=22
)




File fetched successfully from sftp.l3harris.com
